In [33]:
import pandas as pd
import os

In [34]:
years = [2018, 2019, 2020, 2021, 2022, 2023]
years

[2018, 2019, 2020, 2021, 2022, 2023]

In [35]:
for year in years:
    file_path = f'../Project/{year}'

    for file in os.listdir(file_path):
        full_path = os.path.join(file_path, file)
        if not file.endswith(".json") and os.path.isfile(full_path):
            new_name = file + ".json"
            new_full_path = os.path.join(file_path, new_name)
            os.rename(full_path, new_full_path)
            print(f"Renamed the file {file} to {new_name}")

#### Test reading file data 


In [36]:
df = pd.read_json('../Project/2018/201800000.json')

In [37]:
df.shape

(8, 1)